In [1]:
import xarray as xr
import rioxarray as rxr
import shutil
import os
from math import ceil
from tqdm import tqdm

class ChunkWriter():
    def __init__(self, tiff_path: str, save_path: str, chunk_size: int = 100):
        self.da = rxr.open_rasterio(tiff_path)
        self.save_path = save_path
        self.chunk_size = chunk_size

        self.num_y = len(self.da.y)

        self.current_chunk = 0

    def __iter__(self):
        if os.path.exists(self.save_path):
            shutil.rmtree(self.save_path)

        return self

    def __next__(self):
        self.write_chunk(self.current_chunk)

        if (self.current_chunk * self.chunk_size) > self.num_y:
            raise StopIteration
 
        self.current_chunk += 1

    def __len__(self) -> int:
        return ceil(self.num_y / self.chunk_size)

    def write_chunk(self, chunk: int):

        da = self.da.isel(y=slice(chunk * self.chunk_size, (chunk + 1) * self.chunk_size)).load()

        ds = xr.Dataset(
            {
                'rs': da.sel(band=[1, 2, 3, 4]),
                'mask': da.sel(band=5),
                'label': da.sel(band=6),
            }
        )

        ds = ds.transpose('band', 'x', 'y')

        category_dict_reversed = {
            0: "ConstructionSite",
            1: "Building",
            2: "BuildingDistortion",
            3: "GreenAreas",
            4: "RoadAsphalt",
            5: "Forest",
            6: "WaterBasin",
            7: "Path",
            8: "MeadowPasture",
            9: "SealedObjects"
        }

        # Dataset attributes
        ds.attrs = {'creator': 'Julian Kraft'}

        # Variable attributes
        ds['rs'].attrs.update({'source': 'SwissImage RS'})
        ds['label'].attrs.update({'classes': ', '.join([f'{k}={v}' for k, v in category_dict_reversed.items()])})

        ds['rs'] = ds.rs.chunk({'band': 4, 'x': self.chunk_size, 'y': self.chunk_size})
        ds['label'] = ds.label.chunk({'x': self.chunk_size, 'y': self.chunk_size})
        ds['mask'] = ds.mask.chunk({'x': -1, 'y': -1})


        encoding = {}

        for variable in ds.data_vars:
            encoding[variable] = {'compressor': None}

            if variable == 'rs':
                ds[variable] = ds[variable].astype('uint16')

            else:
                ds[variable] = ds[variable].where(ds[variable] != -9999, 255).astype('uint8')

            ds[variable].attrs = {}

        if chunk == 0:
            kwargs = {}
        else:
            for variable in ds.data_vars:
                ds[variable].attrs = {}
            kwargs = {'append_dim': 'y'}

        ds.to_zarr(self.save_path, mode='a', **kwargs)

    def write(self, dev_mode: bool = False):
        for i, _ in enumerate(tqdm(self, ncols=80, desc='Writing chunks')):
            if dev_mode and (i > 1):
                raise StopIteration


In [3]:
chunkwriter = ChunkWriter(
    tiff_path='../data/CombinedData_32signed/CombinedData32signed.tif',
    save_path='../data/combined.zarr',
    chunk_size=500)
chunkwriter.write()

/Users/kraftb/miniforge3/envs/sa2/lib/python3.12/site-packages/pyproj/crs/_cf1x8.py:515: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
Writing chunks: 100%|███████████████████████████| 84/84 [01:18<00:00,  1.07it/s]


In [9]:
xr.open_zarr('../data/combined.zarr').isel(x=0).rs.max().compute()

<xarray.DataArray 'rs' ()> Size: 2B
array(13057, dtype=uint16)
Coordinates:
    spatial_ref  int64 8B 0
    x            float64 8B 2.618e+06

In [29]:
rs_subset.attrs = {}

In [30]:
rs_subset.to_zarr('../data/combined.zarr', mode='a', region={'x': slice(0, 100), 'y': slice(0, 100)})

ValueError: when setting `region` explicitly in to_zarr(), all variables in the dataset to write must have at least one dimension in common with the region's dimensions ['x', 'y'], but that is not the case for some variables here. To drop these variables from this dataset before exporting to zarr, write: .drop_vars(['band', 'spatial_ref'])

In [5]:
ds.to_zarr('../data/combined.zarr', encoding=encoding, mode='w', compute=False)

Delayed('_finalize_store-bad6da60-117e-4e58-bc69-c5323d55aaf9')

In [ ]:


with ProgressBar():
    ds.to_zarr('../data/combined.zarr', encoding=encoding, mode='w')

/Users/kraftb/miniforge3/envs/machflow/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


[########################################] | 100% Completed | 425.35 ms
